In [1]:
import re

from datasets import load_dataset
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch
from PIL import Image


/Users/DocWeaver/.pyenv/versions/3.8.13/envs/marker_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device);

In [31]:
def process_document(image, question):
    # prepare encoder inputs
    pixel_values = processor(image, return_tensors="pt").pixel_values
    print(pixel_values.shape)
    
    # prepare decoder inputs
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
    prompt = task_prompt.replace("{user_input}", question)
    decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids
          
    # generate answer
    outputs = model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )
    
    # postprocess
    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
    
    return processor.token2json(sequence)

In [38]:
file = "../completed_test_scans/Scanned Image.png"
img = Image.open(file)
img = img.convert("RGB")
img = img.resize((400,440),Image.LANCZOS)


In [39]:
img.size

(400, 440)

In [40]:
process_document(img, "name?")

torch.Size([1, 3, 2560, 1920])


{'question': 'name?', 'answer': 'tom'}

In [37]:
process_document(img, "name?")

torch.Size([1, 3, 2560, 1920])


{'question': 'name?', 'answer': 'tom'}

In [12]:
process_document(img, "Is Lemon Juice an Acid or a Base")

{'question': 'Is Lemon Juice an Acid or a Base', 'answer': 'acid'}

In [18]:
process_document(img, "What colour does universal indicator turn in ph 2?")

{'question': 'What colour does universal indicator turn in ph 2?',
 'answer': 'red'}

In [15]:
process_document(img, "ph 2?")

{'question': 'ph 2?', 'answer': 'red'}

In [43]:
pixel_values = processor(img, return_tensors="pt").pixel_values
print(pixel_values.shape)


torch.Size([1, 3, 2560, 1920])


In [44]:
question = "name?"

In [51]:
# prepare decoder inputs
task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
prompt = task_prompt.replace("{user_input}", question)
decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids

In [52]:
  # generate answer
outputs = model.generate(
                    pixel_values.to(device),
                    decoder_input_ids=decoder_input_ids.to(device),
                    max_length=model.decoder.config.max_position_embeddings,
                    early_stopping=True,
                    pad_token_id=processor.tokenizer.pad_token_id,
                    eos_token_id=processor.tokenizer.eos_token_id,
                    use_cache=True,
                    #num_beams=1,
                    bad_words_ids=[[processor.tokenizer.unk_token_id]],
                    return_dict_in_generate=True,               
    )

In [53]:
 # postprocess
sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
processor.token2json(sequence)

{'question': 'name?', 'answer': 'tom'}